<a href="https://colab.research.google.com/github/Meghraj-Webllisto/language-translation/blob/master/17_02fairseq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# working on prepairing training data fo coustom language chinese or japense or korean

In [1]:
cd /content/drive/My Drive/fairseq

/content/drive/My Drive/fairseq


In [2]:
ls

build/              examples/          generate.py     score.py
CODE_OF_CONDUCT.md  fairseq/           hubconf.py      scripts/
CONTRIBUTING.md     fairseq_cli/       interactive.py  setup.py
dist/               fairseq.egg-info/  LICENSE         tests/
docs/               fairseq.gif        preprocess.py   train.py
eval_lm.py          fairseq_logo.png   README.md       validate.py


In [3]:
cd scripts 

/content/drive/My Drive/fairseq/scripts


In [4]:
ls

average_checkpoints.py  count_docs.py        split_train_valid_docs.py
build_sym_alignment.py  __init__.py          spm_decode.py
compare_namespaces.py   read_binarized.py    spm_encode.py
compound_split_bleu.sh  rm_pt.py             spm_train.py
convert_dictionary.lua  sacrebleu_pregen.sh  wav2vec_featurize.py
convert_model.lua       shard_docs.py        wav2vec_manifest.py


In [5]:
cd ..

/content/drive/My Drive/fairseq
